# 1.ติดตั้งไลบรารี

In [ ]:
!pip install --upgrade pip


In [1]:
# Cell 1: ติดตั้งไลบรารี
import sys
!{sys.executable} -m pip install selenium webdriver_manager openpyxl
print("ติดตั้งไลบรารีเสร็จสิ้น (หากติดตั้งไปแล้ว จะแสดงว่า Requirement already satisfied)")

ติดตั้งไลบรารีเสร็จสิ้น (หากติดตั้งไปแล้ว จะแสดงว่า Requirement already satisfied)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import sys
print(sys.executable)

c:\Users\ball-pc\AppData\Local\Programs\Python\Python313\python.exe


# 2.import รหัส

In [2]:
# --- Cell 1: Imports และการตั้งค่า ---
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager


import time
import os
from dotenv import load_dotenv

# โหลดค่าจากไฟล์ .env
load_dotenv()

# --- ค่าคงที่ ---
LOGIN_URL = "https://cenproject.rid.go.th/"
YOUR_USERNAME = os.getenv("CEN_USERNAME")
YOUR_PASSWORD = os.getenv("CEN_PASSWORD")
EXCEL_FILE_PATH = 'table_main_2068.xlsx' # <<<<---- ระบุชื่อไฟล์ Excel ของคุณที่นี่
OUTPUT_EXCEL_PATH = 'result_data.xlsx' # <<<<---- ไฟล์ผลลัพธ์ที่จะถูกสร้างขึ้น

print("การตั้งค่าเสร็จสิ้น")

การตั้งค่าเสร็จสิ้น


In [3]:
print(YOUR_USERNAME, YOUR_PASSWORD)

rid2_0030 Aood2534


# 3.เปิด Chrome เพื่อ Login

In [4]:
# --- Cell 2: เริ่มต้น WebDriver และ Login ---
print("กำลังเริ่มต้น WebDriver...")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.maximize_window()

print(f"กำลังเข้าสู่หน้า Login: {LOGIN_URL}")
driver.get(LOGIN_URL)

wait = WebDriverWait(driver, 5) # เพิ่มเวลารอเป็น 20 วินาทีเผื่อเว็บโหลดช้า

try:
    # 1. Login
    print("กำลังรอหน้า Login...")
    username_field = wait.until(EC.presence_of_element_located((By.NAME, "Username")))
    password_field = wait.until(EC.presence_of_element_located((By.NAME, "Password")))
    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-primary[type='submit']")))

    print("กำลังกรอกข้อมูล Login...")
    username_field.send_keys(YOUR_USERNAME)
    password_field.send_keys(YOUR_PASSWORD)
    login_button.click()
    print("Login สำเร็จ")

    # 2. กดเข้าระบบติดตาม
    print("กำลังรอลิงก์ 'ระบบติดตาม'...")
    system_tracking_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[./img[@alt='banner-track']]")))
    system_tracking_link.click()
    print("คลิก 'ระบบติดตาม' สำเร็จ")

    # --- ส่วนที่แก้ไข ---
    # 3. รอให้หน้ากาก Loading หายไปก่อน
    print("กำลังรอให้หน้า Dashboard โหลดเสร็จ (รอให้ Loading Overlay หายไป)...")
    loading_overlay_xpath = "//div[contains(@style, 'display: block') and contains(@style, 'z-index: 9999')]"
    # เราจะรอจนกว่า Element นี้จะ "มองไม่เห็น" (invisible)
    wait.until(EC.invisibility_of_element_located((By.XPATH, loading_overlay_xpath)))
    print("Loading Overlay หายไปแล้ว หน้าเว็บพร้อมใช้งาน")
except Exception as e:
    print(f"เกิดข้อผิดพลาดระหว่างการ Login หรือการนำทางเริ่มต้น: {e}")
    # driver.quit()

กำลังเริ่มต้น WebDriver...
กำลังเข้าสู่หน้า Login: https://cenproject.rid.go.th/
กำลังรอหน้า Login...
กำลังกรอกข้อมูล Login...
Login สำเร็จ
กำลังรอลิงก์ 'ระบบติดตาม'...
คลิก 'ระบบติดตาม' สำเร็จ
กำลังรอให้หน้า Dashboard โหลดเสร็จ (รอให้ Loading Overlay หายไป)...
Loading Overlay หายไปแล้ว หน้าเว็บพร้อมใช้งาน


# 4.วิ่งเข้าหน้า พรบ 2568

In [27]:

def go_to_main_budget_page(driver):
    try:

        base_budget_url = "https://cenproject.rid.go.th/track/budget?BudgetYear=2025"
        print(f"มาถึงหน้า 'งบประมาณ' หลักแล้ว: {driver.current_url}")

        target_url = base_budget_url + "&BudgetSourceID=1"
        driver.get(target_url)
        print("กำลังรอให้ตารางข้อมูลโครงการโหลด...")

    except Exception as e:
        # แก้ไขข้อความ Error ตามที่ต้องการ
        print("\n--- การนำทางสำเร็จ แต่เกิดข้อผิดพลาดในการรอ Element ---")
        print(f"ข้อผิดพลาด: ไม่สามารถหา Element ที่ต้องการได้ภายในเวลาที่กำหนด")
        print(f"URL ปัจจุบันที่เกิดปัญหา: {driver.current_url}")
        print("สาเหตุที่เป็นไปได้: ID หรือ Class ของตารางข้อมูลในหน้านี้อาจไม่ตรงกับที่ระบุในโค้ด")


In [198]:
go_to_main_budget_page(driver)

มาถึงหน้า 'งบประมาณ' หลักแล้ว: https://cenproject.rid.go.th/track/budget/111?BudgetYear=2025&BudgetSourceID=1&OrganizationName=%E0%B9%82%E0%B8%84%E0%B8%A3%E0%B8%87%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%8A%E0%B8%A5%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%97%E0%B8%B2%E0%B8%99%E0%B9%80%E0%B8%8A%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B8%A3%E0%B8%B2%E0%B8%A2
กำลังรอให้ตารางข้อมูลโครงการโหลด...


### next page 1 kick

In [204]:
# --- [ฟังก์ชันสำหรับคลิกหน้าถัดไป - ฉบับ Reusable] ---
import re
from selenium.webdriver.common.action_chains import ActionChains

def go_to_next_page(driver, wait, table_xpath):
    
    # print("\n--- กำลังตรวจสอบ Pagination ---")
    print(table_xpath)
    try:

        try:
            current_table = wait.until(EC.presence_of_element_located((By.XPATH, table_xpath)))
        except TimeoutException:
            # print(f"!!! ไม่พบตารางตาม XPath ที่ระบุ: {table_xpath} !!!")
            return False

        pagination_text_element = driver.find_element(By.XPATH, "//div[contains(text(), 'แสดงข้อมูล')]")
        pagination_text = pagination_text_element.text
        numbers = re.findall(r'\d+', pagination_text)
        
        can_go_next = False
        if len(numbers) >= 3:
            last_item_shown = int(numbers[1])
            total_items = int(numbers[2])
            if last_item_shown < total_items:
                can_go_next = True
        
        # 3. ตัดสินใจคลิก "Next" ถ้าทำได้
        if can_go_next:
            # print(" -> พบว่ามีหน้าถัดไป, กำลังคลิก 'Next'...")
            
            # ใช้ CSS Selector ที่น่าจะใช้ร่วมกันได้
            next_page_button_selector = "span.mx-4 + button"
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, next_page_button_selector)))
            
            # ใช้ ActionChains เพื่อคลิกเหมือนมนุษย์
            ActionChains(driver).move_to_element(next_button).click().perform()

            # print(" -> เปลี่ยนหน้าสำเร็จ!")
            return True # คืนค่าว่าไปหน้าต่อไปได้
        else:
            print(" -> เป็นหน้าสุดท้ายแล้ว.")
            return False # คืนค่าว่าเป็นหน้าสุดท้าย

    except NoSuchElementException:
        print(" -> ไม่พบข้อความ Pagination, ถือว่าเป็นหน้าสุดท้าย")
        return False
    except Exception as e:
        # print(f"!!! เกิดข้อผิดพลาดร้ายแรงขณะพยายามไปหน้าถัดไป: {e} !!!")
        return False

In [79]:
go_to_next_page(driver, wait, "//table[contains(@class, 'table-zebra')]")  #if returns True,next and returns False if last page


--- กำลังตรวจสอบ Pagination ---
//table[contains(@class, 'table-zebra')]
 -> พบว่ามีหน้าถัดไป, กำลังคลิก 'Next'...
 -> เปลี่ยนหน้าสำเร็จ!


True

### วิ่งเข้ากรอก ผลเบิกจ่าย

In [143]:
df = pd.read_excel("table_main_2568.xlsx")

print(df["ร้อยละ"][:10])


0    100.000000
1     80.847302
2     89.730764
3     99.263834
4    100.000000
5    100.000000
6    100.000000
7    100.000000
8    100.000000
9    100.000000
Name: ร้อยละ, dtype: float64


In [175]:
# --- [โค้ดสำหรับเช็คเงื่อนไข, ค้นหาใน Excel, และคลิกเข้าไปในแต่ละแถว - ฉบับปรับปรุง] ---
# สมมติว่า df และ wait ถูกสร้างไว้แล้ว
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC


# อ่านข้อมูลจาก Excel
df = pd.read_excel("table_main_2568.xlsx")


def extract_budget_data(driver, wait):
    # ดึงชื่อ งานจาก xpath/html/body/div[2]/div[1]/main/main/div[1]
    xpath_get_name = "//html/body/div[2]/div[1]/main/main/div[2]"
    name_element = wait.until(EC.presence_of_element_located((By.XPATH, xpath_get_name)))
    name = name_element.text.strip()
    print(f"รายการงาน: {name}...")
    i = 0 # ตัวนับแถว
    count_items = 0
    list_of_under100 = []  # รายการงานที่มีเปอร์เซ็นต์น้อยกว่า 100%




    while True:
        # print(f"\n--- กำลังประมวลผลแถวที่ {i + 1} ---")
        
        try:
            # 1. หาจำนวนแถวทั้งหมดในหน้าปัจจุบัน
            tbody = wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-zebra')]//tbody")))
            subtask_rows_count = len(tbody.find_elements(By.XPATH, ".//tr"))
            # print(f"พบ {subtask_rows_count} งานย่อยในหน่วยงานนี้. กำลังตรวจสอบเงื่อนไข...")

            for i in range(subtask_rows_count):
                # ใช้ i+1 เพราะ index ของ Python เริ่มที่ 0 แต่ XPath เริ่มที่ 1
                print(f"\n--- กำลังตรวจสอบแถวที่ {i + 1} ---")

                try:
                    current_row_xpath = f"(//table[contains(@class, 'table-zebra')]//tbody/tr)[{i + 1}]"
                    current_row = wait.until(EC.presence_of_element_located((By.XPATH, current_row_xpath)))
                    subtask_percent_text  = current_row.find_element(By.XPATH, ".//td[9]").text.strip()
                    subtask_name_text = current_row.find_element(By.XPATH, ".//td[2]").text.strip()
                    print(f" -> ชื่อ: {subtask_name_text}, เปอร์เซ็นต์: {subtask_percent_text}")

                    if float(subtask_percent_text) < 100:
                         
                        matched = df[df['ชื่อ / ประมาณการ'] == subtask_name_text]
                        if not matched.empty:
                            project_name = matched.iloc[0]['ชื่อ / ประมาณการ']
                            percent_excel = round(matched.iloc[0]['ร้อยละ'], 2)
                            print(f" -> ชื่อ excel: {project_name}, เปอร์เซ็นต์: {percent_excel}")

                            
                            
                            list_of_under100.append(subtask_name_text)
                        

                        # if i == max(0, subtask_rows_count - 1):
                        #     if not go_to_next_page(driver, wait, "//table[contains(@class, 'table-zebra')]"):
                        #         print(f" -> ไม่สามารถไปหน้าถัดไปได้,รายการทั้งหมดของ {name} {len(list_of_under100)} รายการงาน ออกจากลูป")
                        #         return
                        #     i = -1


                        


                                      
                        





                    
                
                except TimeoutException:
                    print(f"!!! ไม่พบแถวที่ {i + 1} !!!")
                    continue
                except NoSuchElementException:
                    print(f"!!! ไม่พบแถวที่ {i + 1} !!!")
                    continue

            
        
        except Exception as e:
            print(f"เกิดข้อผิดพลาดในการค้นหาตาราง: {e}")
        break

In [207]:
go_to_main_budget_page(driver)

มาถึงหน้า 'งบประมาณ' หลักแล้ว: https://cenproject.rid.go.th/track/budget/275?BudgetYear=2025&BudgetSourceID=1&OrganizationName=%E0%B9%82%E0%B8%84%E0%B8%A3%E0%B8%87%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%81%E0%B9%88%E0%B8%AD%E0%B8%AA%E0%B8%A3%E0%B9%89%E0%B8%B2%E0%B8%87%E0%B8%8A%E0%B8%A5%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%97%E0%B8%B2%E0%B8%99%E0%B8%97%E0%B8%B5%E0%B9%88+2
กำลังรอให้ตารางข้อมูลโครงการโหลด...


In [205]:
go_to_next_page(driver, wait, "//table[contains(@class, 'table-zebra')]")

//table[contains(@class, 'table-zebra')]
 -> เป็นหน้าสุดท้ายแล้ว.


False

In [206]:
extract_budget_data(driver, wait)


รายการงาน: หน่วยงาน: โครงการก่อสร้างชลประทานที่ 2...

--- กำลังตรวจสอบแถวที่ 1 ---
 -> ชื่อ: (197) ฝายขำพร้อมระบบส่งน้ำ ตำบลงิม อำเภอปง จังหวัดพะเยา - [ 10336 ], เปอร์เซ็นต์: 89.08
 -> ชื่อ excel: (197) ฝายขำพร้อมระบบส่งน้ำ ตำบลงิม อำเภอปง จังหวัดพะเยา - [ 10336 ], เปอร์เซ็นต์: 89.08

--- กำลังตรวจสอบแถวที่ 2 ---
 -> ชื่อ: (176) ฝายบ่อเป๊กพร้อมระบบส่งน้ำ ตำบลเรือง อำเภอเมืองน่าน จังหวัดน่าน - [ 10315 ], เปอร์เซ็นต์: 79.86
 -> ชื่อ excel: (176) ฝายบ่อเป๊กพร้อมระบบส่งน้ำ ตำบลเรือง อำเภอเมืองน่าน จังหวัดน่าน - [ 10315 ], เปอร์เซ็นต์: 79.84

--- กำลังตรวจสอบแถวที่ 3 ---
 -> ชื่อ: (175) ฝายห้วยฮ้อมพร้อมระบบส่งน้ำ ตำบลกองควาย อำเภอเมืองน่าน จังหวัดน่าน - [ 10314 ], เปอร์เซ็นต์: 80.06
 -> ชื่อ excel: (175) ฝายห้วยฮ้อมพร้อมระบบส่งน้ำ ตำบลกองควาย อำเภอเมืองน่าน จังหวัดน่าน - [ 10314 ], เปอร์เซ็นต์: 80.06

--- กำลังตรวจสอบแถวที่ 4 ---
 -> ชื่อ: (174) ฝายห้วยแอบพร้อมระบบส่งน้ำ ตำบลศรีษะเกษ อำเภอนาน้อย จังหวัดน่าน - [ 10313 ], เปอร์เซ็นต์: 86.18
 -> ชื่อ excel: (174) ฝายห้วยแอบพร้อมระบบส่งน้ำ ตำบล

In [138]:
import sys
from contextlib import redirect_stdout

with open("output_extract.text", "w", encoding="utf-8") as f:
    with redirect_stdout(f):
        extract_budget_data(driver, wait)


KeyboardInterrupt: 

### วิ่งกลับหน้า รวมตาราง ของโครงการ

In [12]:
# --- ใส่โค้ดนี้แทนที่ try...except บล็อกเดิมของคุณ ---
def go_to_subtask(driver,name):
    try:
        base_sub_budget_url = "https://cenproject.rid.go.th/track/budget/111?BudgetYear=2025&BudgetSourceID=1&OrganizationName="
        print(f"มาถึงหน้า 'งบประมาณ' หลักแล้ว: {driver.current_url}")

        target_url = base_sub_budget_url + name
        driver.get(target_url)
    except Exception as e:
        print(f"URL ปัจจุบันที่เกิดปัญหา: {driver.current_url}")



In [13]:
go_to_subtask(driver, "โครงการชลประทานเชียงราย")

มาถึงหน้า 'งบประมาณ' หลักแล้ว: https://cenproject.rid.go.th/track/budget?BudgetYear=2025&BudgetSourceID=1
